In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import *
from keras import Model, Sequential, Input
from keras import optimizers
import keras.backend as K
import numpy as np
from pprint import pprint

/home/henrik/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_style_model():
    model = load_model('darknet.h5')
    inputs = model.layers[0].input
    x = model.layers[16].output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs, x)


def EuclSqDist(inputs):
    return K.mean(K.square(inputs[0]- inputs[1]), axis=-1)


def EuclSqLambda():
    return Lambda(EuclSqDist, output_shape=(1,))

In [5]:
base_model = make_style_model()
input_a = Input((256,256,3))
input_b = Input((256,256,3))
encoding_a = base_model(input_a)
encoding_b = base_model(input_b)
dist = EuclSqLambda()([encoding_a, encoding_b])
model = Model([input_a, input_b], dist)
model.summary()

/home/henrik/miniconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_5 (Model)                 (None, 128)          4292592     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1)            0           model_5[1][0]                    
          

In [ ]:
model.compile()